In [6]:
from qdrant_client import QdrantClient

In [7]:
# Connect to Qdrant
qdrant_client = QdrantClient(
    "https://3ad0d7d5-4015-4ccc-b8d5-c09f8121bd2b.us-east4-0.gcp.cloud.qdrant.io",
    api_key="GP2xb22FrO1cMMBBcaxkGJHzWHBBf86yms7u48Sxjco1LoENfXlo8Q",
)

qdrant_client.query(
    collection_name="mydata_content",
    query_text="Who likes taking pictures?"
)

[QueryResponse(id='4cb66f4a-8f20-5363-9fc7-322ad5fdc15d', embedding=None, metadata={'_dlt_id': 'XAS0KiLsPAOnFQ', '_dlt_load_id': '1701704878.659247', 'backstory': 'A scientist by day, a virtual adventurer by night', 'city': 'South Tracy', 'email': 'smckay@yahoo.com', 'favorite_gaming_moment': 'Recalls the epic win in the final seconds of a battle royale game', 'hobbies': 'Loves hiking and photography', 'in_game_behavior': 'Diplomatic and a team player, excels in cooperative tasks', 'name': 'Brent Robinson', 'personality_description': 'Competitive spirit, always aiming for the top scores', 'rfam_acc': 'RF00004'}, document='', score=0.80612046),
 QueryResponse(id='bde05b88-a367-5786-b779-40c85788433f', embedding=None, metadata={'_dlt_id': 'o2K6zysnQrqW3Q', '_dlt_load_id': '1701704878.659247', 'backstory': 'A retired superhero, finds solace in virtual battles', 'city': 'Lake Richard', 'email': 'vanessa12@russell-reynolds.com', 'favorite_gaming_moment': 'Fondly remembers building an entire

In [12]:
import openai
import logging

In [5]:
async def generate_chatgpt_output(query: str, context: str = None, api_key=None, model_name="gpt-3.5-turbo"):
    """
    Generate a response from the OpenAI ChatGPT model.

    Args:
        query (str): The user's query or message.
        context (str, optional): Additional context for the conversation. Defaults to an empty string.
        api_key (str, optional): Your OpenAI API key. If not provided, the globally configured API key will be used.
        model_name (str, optional): The name of the ChatGPT model to use. Defaults to "gpt-3.5-turbo".

    Returns:
        str: The response generated by the ChatGPT model.

    Raises:
        Exception: If an error occurs during the API call, an error message is returned for the caller to handle.
    """
    if not context:
        context = ""

    messages = [
        {"role": "system", "content": "You are a helpful assistant."},
        {"role": "assistant", "content": context},
        {"role": "user", "content": query},
    ]

    try:
        openai.api_key = api_key if api_key else openai.api_key  # Use the provided API key or the one set globally

        response = openai.ChatCompletion.create(
            model=model_name,
            messages=messages,
        )

        llm_output = response.choices[0].message.content
        return llm_output
    except Exception as e:
        return f"An error occurred: {e}"  # Return the error message for the caller to handle

In [3]:
from deepeval.test_case import LLMTestCase

async def eval_test(
    query=None,
    output=None,
    expected_output=None,
    context=None,
    synthetic_test_set=False,
):
    logging.info("Generating chatgpt output")
    result_output = await generate_chatgpt_output(query, str(context))

    logging.info("Moving on")

    if synthetic_test_set:
        test_case = synthetic_test_set
    else:
        test_case = LLMTestCase(
            input=str(query),
            actual_output=str(result_output),
            expected_output=str(expected_output),
            context=[str(context)],
        )
    metric = OverallScoreMetric()

    # If you want to run the test
    test_result = run_test(test_case, metrics=[metric], raise_error=False)

    def test_result_to_dict(test_result):
        return {
            "success": test_result.success,
            "score": test_result.score,
            "metric_name": test_result.metric_name,
            "query": test_result.query,
            "output": test_result.output,
            "expected_output": test_result.expected_output,
            "metadata": test_result.metadata,
            "context": test_result.context,
        }

    test_result_dict = []
    for test in test_result:
        test_result_it = test_result_to_dict(test)
        test_result_dict.append(test_result_it)
    return test_result_dict

In [ ]:
async def run_eval(test_item, search_result):
    logging.info("Initiated test set evaluation")
    test_eval = await eval_test(
        query=str(test_item["question"]),
        expected_output=str(test_item["answer"]),
        context=str(search_result),
    )
    logging.info("Successfully evaluated test set")
    return test_eval